In [3]:
import laser_lib
import numpy as np
import math
import time


In [4]:
queue = laser_lib.DacQueue()

Found  1 Helios DACs


In [13]:
def calculate_sample_energy(sample):
    # Calculate the energy of the sample
    energy = np.sum(np.square(sample))
    return energy

def energy_wave(sample):
    # Calculate the energy of the sample
    energy = np.sum(np.square(sample))
    return energy

def stream_bytes_to_numpy(data, num_channels):
     # Convert bytes to a NumPy array of floats
    audio_data = np.frombuffer(data, dtype=np.int16)
    
    # Normalize the audio data to the range [-1, 1]
    audio_data = audio_data.astype(np.float32) / np.iinfo(np.int16).max
    
    # Reshape the data to have one row per channel
    audio_data = audio_data.reshape(-1, num_channels)

    return audio_data

4096
(1024, 2)


ALSA lib pcm_dsnoop.c:601:(snd_pcm_dsnoop_open) unable to open slave
ALSA lib pcm_dmix.c:1032:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_oss.c:397:(_snd_pcm_oss_open) Cannot open device /dev/dsp
ALSA lib pcm_oss.c:397:(_snd_pcm_oss_open) Cannot open device /dev/dsp
ALSA lib confmisc.c:160:(snd_config_get_card) Invalid field card
ALSA lib pcm_usb_stream.c:482:(_snd_pcm_usb_stream_open) Invalid card 'card'
ALSA lib confmisc.c:160:(snd_config_get_card) Invalid field card
ALSA lib pcm_usb_stream.c:482:(_snd_pcm_usb_stream_open) Invalid card 'card'
ALSA lib pcm_dmix.c:1032:(snd_pcm_dmix_open) unable to open slave


In [ ]:
# Integrate the music with the laser
import wave
import pyaudio

T = 200 # at 30kpps this is 30x a second
color_arr = np.ones((T,3))
theta = np.linspace(0, 2*np.pi, T)
freq = 2
pos_arr = np.zeros((T, 2))
pos_arr[:, 0] = np.cos(freq*theta)
pos_arr[:, 1] = np.sin(freq*theta)

def play_audio_file(file_path, chunk_size=2*1024):
    wf = wave.open(file_path, 'rb')
    p = pyaudio.PyAudio()

    num_channels = wf.getnchannels()
    # Open output stream
    stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                    channels=num_channels,
                    rate=wf.getframerate(),
                    output=True)

    data = wf.readframes(chunk_size)


    max_energy = 0
    # Play the file chunk by chunk
    while data:
        stream.write(data)
        data = wf.readframes(chunk_size)
        energy = calculate_sample_energy(stream_bytes_to_numpy(data, num_channels))
        if(energy > max_energy):
            max_energy = energy
        energy_normed = energy/max_energy
        queue.submit(energy_normed*pos_arr, color_arr, debug=False)
        

    # Close the stream and terminate PyAudio
    stream.close()
    p.terminate()

file_path = "PurpleSun.wav"
play_audio_file(file_path)


In [ ]:
# Play music chunk by chunk
import wave
import pyaudio

def play_audio_file(file_path, chunk_size=1024):
    wf = wave.open(file_path, 'rb')
    p = pyaudio.PyAudio()

    num_channels = wf.getnchannels()
    # Open output stream
    stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                    channels=num_channels,
                    rate=wf.getframerate(),
                    output=True)

    data = wf.readframes(chunk_size)

        # Convert bytes to a NumPy array of floats
    audio_data = np.frombuffer(data, dtype=np.int16)
    
    # Normalize the audio data to the range [-1, 1]
    audio_data = audio_data.astype(np.float32) / np.iinfo(np.int16).max
    
    # Reshape the data to have one row per channel
    audio_data = audio_data.reshape(-1, num_channels)

    print(len(data))
    print(audio_data.shape)
    return

    # Play the file chunk by chunk
    while data:
        stream.write(data)
        data = wf.readframes(chunk_size)

    # Close the stream and terminate PyAudio
    stream.close()
    p.terminate()

file_path = "PurpleSun.wav"
play_audio_file(file_path)
